#  Rosemann Store Slaes Prediction from August to January 2015

## Import necessary modules for exploration

In [14]:
import pandas as pd


## Load the dataset(s) and view the first rows


In [40]:
df = pd.read_csv('rossmann-store-sales/train.csv')
df_store = pd.read_csv('rossmann-store-sales/store.csv')
full_store_details = df.merge(df_store)
full_store_details.tail(70)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
1017139,1115,1,2013-03-11,5503,362,1,0,0,0,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec"
1017140,1115,7,2013-03-10,0,0,0,0,0,0,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec"
1017141,1115,6,2013-03-09,5444,410,1,0,0,0,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec"
1017142,1115,5,2013-03-08,5626,403,1,1,0,0,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec"
1017143,1115,4,2013-03-07,5716,403,1,1,0,0,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1115,6,2013-01-05,4771,339,1,0,0,1,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec"
1017205,1115,5,2013-01-04,4540,326,1,0,0,1,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec"
1017206,1115,4,2013-01-03,4297,300,1,0,0,1,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec"
1017207,1115,3,2013-01-02,3697,305,1,0,0,1,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec"


In a data frame each row corresponds to one observation (e.g., a store) and
each column corresponds to one feature (type, promotions run, assortment.). 

For example, by looking at the first observation we can see that store id 1 is of type 'c' and carries a basic assortment of goods (denoted as a) and is 1270 metres from the next competitor that was opened around September in 2008. Additionally, this store does not participate in the ongoing promotion (labelled Promo 2).


## Get to know your data

In [42]:
full_store_details.dtypes , full_store_details.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1017209 entries, 0 to 1017208
Data columns (total 18 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Store                      1017209 non-null  int64  
 1   DayOfWeek                  1017209 non-null  int64  
 2   Date                       1017209 non-null  object 
 3   Sales                      1017209 non-null  int64  
 4   Customers                  1017209 non-null  int64  
 5   Open                       1017209 non-null  int64  
 6   Promo                      1017209 non-null  int64  
 7   StateHoliday               1017209 non-null  object 
 8   SchoolHoliday              1017209 non-null  int64  
 9   StoreType                  1017209 non-null  object 
 10  Assortment                 1017209 non-null  object 
 11  CompetitionDistance        1014567 non-null  float64
 12  CompetitionOpenSinceMonth  693861 non-null   float64
 13  CompetitionO

(Store                          int64
 DayOfWeek                      int64
 Date                          object
 Sales                          int64
 Customers                      int64
 Open                           int64
 Promo                          int64
 StateHoliday                  object
 SchoolHoliday                  int64
 StoreType                     object
 Assortment                    object
 CompetitionDistance          float64
 CompetitionOpenSinceMonth    float64
 CompetitionOpenSinceYear     float64
 Promo2                         int64
 Promo2SinceWeek              float64
 Promo2SinceYear              float64
 PromoInterval                 object
 dtype: object,
 None)

Enter description of data here..

## Wrangle some data

Since we have discovered that non-participating stores in the Promo2 yield NAN values in the *Promo2SinceWeek 	* , *Promo2SinceYear* and *PromoInterval* columns it is possible to adjust for this without skewing our data by filling in the NAN values using a pipeline

The first step in building the pipeline is to define each transformer type. The convention here is generally to create transformers for the different variable types.

## Import modules necessary for this task

In [51]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier

In [59]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='0')),
    ('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='NaN')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])


Next we use the ColumnTransformer to apply the transformations to the correct columns in the dataframe. Before building this I have stored lists of the numeric and categorical columns using the pandas dtype method.

In [49]:
numeric_features = full_store_details[['CompetitionOpenSinceYear' , 'CompetitionOpenSinceMonth']]

In [60]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)
    ])
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='zero')),
                                                 ('scaler', StandardScaler())]),
                                          CompetitionOpenSinceYear  CompetitionOpenSinceMonth
0                          2008.0                        9.0
1                          2008.0                        9.0
2                          2008.0                        9.0
3                          2008.0                        9.0
4                          2008.0                        9.0
...                           ...                        ...
1017204                       NaN                        NaN
1017205                       NaN                        NaN
1017206                       NaN                        NaN
1017207                       NaN                        NaN
1017208                       N

In [ ]:
rf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier())])